# Introduction

![](https://media.giphy.com/media/UUsOy6IWmzw6mmeOpQ/giphy.gif)

The first case of the 2019–20 coronavirus pandemic in India was reported on 30 January 2020, originating from China. Experts suggest the number of infections could be much higher as India's testing rates are among the lowest in the world. The infection rate of COVID-19 in India is reported to be 1.7, significantly lower than in the worst affected countries.

The outbreak has been declared an epidemic in more than a dozen states and union territories, where provisions of the Epidemic Diseases Act, 1897 have been invoked, and educational institutions and many commercial establishments have been shut down. India has suspended all tourist visas, as a majority of the confirmed cases were linked to other countries.

On 22 March 2020, India observed a 14-hour voluntary public curfew at the instance of the prime minister Narendra Modi. The government followed it up with lockdowns in 75 districts where COVID cases had occurred as well as all major cities. Further, on 24 March, the prime minister ordered a nationwide lockdown for 21 days, affecting the entire 1.3 billion population of India.

The World Health Organisation chief executive director of health emergencies programme Michael Ryan said that India had "tremendous capacity" to deal with the coronavirus outbreak and, as the second most populous country, will have enormous impact on the world's ability to deal with it.Other commentators worried about the economic devastation caused by the lockdown, which has huge effects on informal workers, micro and small enterprises, farmers and the self-employed, who are left with no livelihood in the absence of transportation and access to markets. The lockdown was justified by the government and other agencies for being preemptive to prevent India from entering a higher stage which could make handling very difficult and cause even more losses thereafter.

According to a study at Shiv Nadar University, India could have witnessed a surge of 31,000 cases of disease between March 24 and April 14 without lockdown.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.integrate import odeint

from plotly.offline import iplot, init_notebook_mode
import math
import bokeh 
import matplotlib.pyplot as plt
import plotly.express as px
from urllib.request import urlopen
import json
from dateutil import parser
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import row, column
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span
import warnings
warnings.filterwarnings("ignore")
output_notebook(resources=INLINE)

# Global

In [ ]:
country_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
country_codes = country_codes.drop('GDP (BILLIONS)', 1)
country_codes.rename(columns={'COUNTRY': 'Country', 'CODE': 'Code'}, inplace=True)

In [ ]:
virus_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

prev_index = 0
first_time = False
tmp = 0


for i, row in virus_data.iterrows():

    if(virus_data.loc[i,'SNo'] < 1342 and virus_data.loc[i,'Province/State']=='Hubei'):
        if(first_time):
            tmp = virus_data.loc[i,'Confirmed']
            prev_index = i
            virus_data.loc[i,'Confirmed'] = virus_data.loc[i,'Confirmed'] + 593
            first_time = False
        else:
            increment = virus_data.loc[i,'Confirmed'] - tmp
            tmp = virus_data.loc[i,'Confirmed']
            virus_data.loc[i,'Confirmed'] = virus_data.loc[prev_index,'Confirmed'] + increment + 593
            prev_index = i
    

virus_data.rename(columns={'Country/Region': 'Country', 'ObservationDate': 'Date'}, inplace=True)
virus_data = virus_data.fillna('unknow')
virus_data['Country'] = virus_data['Country'].str.replace('US','United States')
virus_data['Country'] = virus_data['Country'].str.replace('UK','United Kingdom') 
virus_data['Country'] = virus_data['Country'].str.replace('Mainland China','China')
virus_data['Country'] = virus_data['Country'].str.replace('South Korea','Korea, South')
virus_data['Country'] = virus_data['Country'].str.replace('North Korea','Korea, North')
virus_data['Country'] = virus_data['Country'].str.replace('Macau','China')
virus_data['Country'] = virus_data['Country'].str.replace('Ivory Coast','Cote d\'Ivoire')
virus_data = pd.merge(virus_data,country_codes,on=['Country'])
#virus_data.head()
#print(len(virus_data))

In [ ]:
top_country = virus_data.loc[virus_data['Date'] == virus_data['Date'].iloc[-1]]
top_country = top_country.groupby(['Code','Country'])['Confirmed'].sum().reset_index()
top_country = top_country.sort_values('Confirmed', ascending=False)
top_country = top_country[:30]
top_country_codes = top_country['Country']
top_country_codes = list(top_country_codes)

#countries = virus_data.loc[virus_data['Country'] in top_country_codes]
countries = virus_data[virus_data['Country'].isin(top_country_codes)]
countries_day = countries.groupby(['Date','Code','Country'])['Confirmed','Deaths','Recovered'].sum().reset_index()


exponential_line_x = []
exponential_line_y = []
for i in range(16):
    exponential_line_x.append(i)
    exponential_line_y.append(i)
    
## TOP 30 ##
"""
china = countries_day.loc[countries_day['Code']=='CHN']

new_confirmed_cases_china = []
new_confirmed_cases_china.append( list(china['Confirmed'])[0] - list(china['Deaths'])[0] 
                           - list(china['Recovered'])[0] )

for i in range(1,len(china)):

    new_confirmed_cases_china.append( list(china['Confirmed'])[i] - 
                                     list(china['Deaths'])[i] - 
                                     list(china['Recovered'])[i])
    
"""
italy = countries_day.loc[countries_day['Code']=='ITA']

new_confirmed_cases_ita = []
new_confirmed_cases_ita.append( list(italy['Confirmed'])[0] - list(italy['Deaths'])[0] 
                           - list(italy['Recovered'])[0] )

for i in range(1,len(italy)):
    
    new_confirmed_cases_ita.append( list(italy['Confirmed'])[i] - 
                                  list(italy['Deaths'])[i] - 
                                  list(italy['Recovered'])[i])
    
    
india = countries_day.loc[countries_day['Code']=='IND']

new_confirmed_cases_india = []
new_confirmed_cases_india.append( list(india['Confirmed'])[0] - list(india['Deaths'])[0] 
                           - list(india['Recovered'])[0] )

for i in range(1,len(india)):
    
    new_confirmed_cases_india.append( list(india['Confirmed'])[i] - 
                                     list(india['Deaths'])[i] - 
                                    list(india['Recovered'])[i])
    

spain = countries_day.loc[countries_day['Code']=='ESP']

new_confirmed_cases_spain = []
new_confirmed_cases_spain.append( list(spain['Confirmed'])[0] - list(spain['Deaths'])[0] 
                           - list(spain['Recovered'])[0] )

for i in range(1,len(spain)):
    
    new_confirmed_cases_spain.append( list(spain['Confirmed'])[i] - 
                                     list(spain['Deaths'])[i] - 
                                    list(spain['Recovered'])[i])
    

us = countries_day.loc[countries_day['Code']=='USA']

new_confirmed_cases_us = []
new_confirmed_cases_us.append( list(us['Confirmed'])[0] - list(us['Deaths'])[0] 
                           - list(us['Recovered'])[0] )

for i in range(1,len(us)):
    
    new_confirmed_cases_us.append( list(us['Confirmed'])[i] - 
                                     list(us['Deaths'])[i] - 
                                    list(us['Recovered'])[i])
    
    
german = countries_day.loc[countries_day['Code']=='DEU']

new_confirmed_cases_german = []
new_confirmed_cases_german.append( list(german['Confirmed'])[0] - list(german['Deaths'])[0] 
                           - list(german['Recovered'])[0] )

for i in range(1,len(german)):
    
    new_confirmed_cases_german.append( list(german['Confirmed'])[i] - 
                                     list(german['Deaths'])[i] - 
                                    list(german['Recovered'])[i])
    
p1 = figure(plot_width=800, plot_height=550, title="Trajectory of Covid-19")
p1.grid.grid_line_alpha=0.3
p1.ygrid.band_fill_color = "olive"
p1.ygrid.band_fill_alpha = 0.1
p1.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p1.yaxis.axis_label = 'New confirmed cases (Log scale)'

p1.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=0.5)

#p1.line(np.log(list(china['Confirmed'])), np.log(new_confirmed_cases_china), color='#DBAE23', 
#        legend_label='China', line_width=1)
#p1.circle(np.log(list(china['Confirmed'])[-1]), np.log(new_confirmed_cases_china[-1]), fill_color="white", size=5)

p1.line(np.log(list(italy['Confirmed'])), np.log(new_confirmed_cases_ita), color='#3EC358', 
        legend_label='Italy', line_width=1)
p1.circle(np.log(list(italy['Confirmed'])[-1]), np.log(new_confirmed_cases_ita[-1]), fill_color="white", size=5)

#p1.line(np.log(list(corea_s['Confirmed'])), np.log(new_confirmed_cases_corea), color='#C3893E', 
#       legend_label='South Korea', line_width=1)
#p1.circle(np.log(list(corea_s['Confirmed'])[-1]), np.log(new_confirmed_cases_corea[-1]), fill_color="white", size=5)


p1.line(np.log(list(india['Confirmed'])), np.log(new_confirmed_cases_india), color='#3E4CC3', 
        legend_label='India', line_width=1)
p1.circle(np.log(list(india['Confirmed'])[-1]), np.log(new_confirmed_cases_india[-1]), fill_color="white", size=5)

p1.line(np.log(list(spain['Confirmed'])), np.log(new_confirmed_cases_spain), color='#F54138', 
        legend_label='Spain', line_width=1)
p1.circle(np.log(list(spain['Confirmed'])[-1]), np.log(new_confirmed_cases_spain[-1]), fill_color="white", size=5)

p1.line(np.log(list(us['Confirmed'])), np.log(new_confirmed_cases_us), color='#23BCDB', 
        legend_label='United States', line_width=1)
p1.circle(np.log(list(us['Confirmed'])[-1]), np.log(new_confirmed_cases_us[-1]), fill_color="white", size=5)

p1.line(np.log(list(german['Confirmed'])), np.log(new_confirmed_cases_german), color='#010A0C', 
        legend_label='Germany', line_width=1)
p1.circle(np.log(list(german['Confirmed'])[-1]), np.log(new_confirmed_cases_german[-1]), fill_color="white", size=5)

p1.legend.location = "bottom_right"

output_file("coronavirus.html", title="coronavirus.py")

show(p1)

In [ ]:
countries = virus_data[virus_data['Country'].isin(top_country_codes)]
countries_day = countries.groupby(['Date','Code','Country'])['Confirmed','Deaths','Recovered'].sum().reset_index()


exponential_line_x = []
exponential_line_y = []
for i in range(16):
    exponential_line_x.append(i)
    exponential_line_y.append(i)
       
india = countries_day.loc[countries_day['Code']=='IND']

new_confirmed_cases_india = []
new_confirmed_cases_india.append( list(india['Confirmed'])[0] - list(india['Deaths'])[0] 
                           - list(india['Recovered'])[0] )

for i in range(1,len(india)):
    
    new_confirmed_cases_india.append( list(india['Confirmed'])[i] - 
                                     list(india['Deaths'])[i] - 
                                    list(india['Recovered'])[i])
    
 
p1 = figure(plot_width=800, plot_height=550, title="Trajectory of Covid-19 in India")
p1.grid.grid_line_alpha=0.3
p1.ygrid.band_fill_color = "olive"
p1.ygrid.band_fill_alpha = 0.1
p1.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p1.yaxis.axis_label = 'New confirmed cases (Log scale)'
p = figure(plot_width=400, plot_height=400)
p.outline_line_width = 7
p.outline_line_alpha = 0.3
p.outline_line_color = "navy"

p1.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=0.5)

p1.line(np.log(list(india['Confirmed'])), np.log(new_confirmed_cases_india), color='#3E4CC3', 
        legend_label='India', line_width=1)
p1.circle(np.log(list(india['Confirmed'])[-1]), np.log(new_confirmed_cases_india[-1]), fill_color="white", size=5)

p1.legend.location = "bottom_right"

output_file("coronavirus_india.html", title="India.py")

show(p1)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/Trakectproes-Covid-confirmed-cases-since-100th-case.svg/800px-Trakectproes-Covid-confirmed-cases-since-100th-case.svg.png)

In [ ]:
import plotly as py
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=False)

# Time plot 

In [ ]:
corona_data=pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')
choro_map=px.choropleth(corona_data, 
                    locations="Country/Region", 
                    locationmode = "country names",
                    color="Confirmed", 
                    hover_name="Country/Region", 
                    animation_frame="ObservationDate"
                   )

choro_map.update_layout(
    title_text = 'Global Spread of Coronavirus',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
choro_map.show()

# The Story of COVID-19 in India

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/COVID-19_Outbreak_Cases_in_India.svg/512px-COVID-19_Outbreak_Cases_in_India.svg.png)

# Containment Zones in India

In [ ]:
import pandas as pd
import numpy as np
zone=pd.read_csv('/kaggle/input/covid-19-india-zone-classification/lockdownindiawarningzones.csv')

In [ ]:
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]}')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
    
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
plotPerColumnDistribution(zone, 10, 5)

![](https://images.livemint.com/img/2020/04/19/600x338/newheaderwithjkandts_1587287652517_1587287652901.png)

# Tree plot

In [ ]:
covid_India_cases = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
covid_India_cases.rename(columns={'State/UnionTerritory': 'State', 'Cured': 'Recovered', 'Confirmed': 'Confirmed'}, inplace=True)

statewise_cases = pd.DataFrame(covid_India_cases.groupby(['State'])['Confirmed', 'Deaths', 'Recovered'].max().reset_index())
statewise_cases["Country"] = "India" # in order to have a single root node
fig = px.treemap(statewise_cases, path=['Country','State'], values='Confirmed',
                  color='Confirmed', hover_data=['State'],
                  color_continuous_scale='Rainbow')
fig.show()

# Time Plot

In [ ]:
import IPython
IPython.display.HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1977187" data-url="https://flo.uri.sh/visualisation/1977187/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

# Map View

In [ ]:
covid_India_cases = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
covid_India_cases.rename(columns={'State/UnionTerritory': 'State', 'Cured': 'Recovered', 'Confirmed': 'Confirmed'}, inplace=True)

statewise_cases = pd.DataFrame(covid_India_cases.groupby(['State'])['Confirmed', 'Deaths', 'Recovered'].max().reset_index())
last=statewise_cases
pos=pd.read_csv('../input/utm-of-india/UTM ZONES of INDIA.csv')
ind_grp=last.merge(pos , left_on='State', right_on='State / Union Territory')

In [ ]:
import folium
map = folium.Map(location=[20.5937, 78.9629], zoom_start=4,tiles='cartodbpositron')

for lat, lon,state,Confirmed,Recovered,Deaths in zip(ind_grp['Latitude'], ind_grp['Longitude'],ind_grp['State'],ind_grp['Confirmed'],ind_grp['Recovered'],ind_grp['Deaths']):
    folium.CircleMarker([lat, lon],
                        radius=5,
                        color='YlOrRd',
                      popup =(
                    'State: ' + str(state) + '<br>'
                    'Confirmed: ' + str(Confirmed) + '<br>'
                      'Recovered: ' + str(Recovered) + '<br>'
                      'Deaths: ' + str(Deaths) + '<br>'),

                        fill_color='red',
                        fill_opacity=0.7 ).add_to(map)
map

# Heat Map 

In [ ]:
from folium.plugins import HeatMap
m = folium.Map(location = [20.5937, 78.9629], zoom_start = 4,tiles='cartodbpositron',columns = ['State/UnionTerritory','Confirmed'],)

heat_data = [[row['Latitude'],row['Longitude']] for index, row in ind_grp.iterrows()]
HeatMap(heat_data,radius=16.5, blur = 5.5).add_to(m)

m

In [ ]:
ind_map=pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
#ind_map.head()
pos=pd.read_csv('../input/utm-of-india/UTM ZONES of INDIA.csv')
ind_map1=ind_map.merge(pos , left_on='State/UnionTerritory', right_on='State / Union Territory')
#ind_map1.head()

In [ ]:
#ind_map = ind_grp
ind_map1  = ind_map1.groupby(['Date', 'State/UnionTerritory','Latitude','Longitude'])['Confirmed'].sum()


ind_map1 = ind_map1.reset_index()
ind_map1.head()
ind_map1['size'] = ind_map1['Confirmed']*90000000
ind_map1
fig = px.scatter_mapbox(ind_map1, lat="Latitude", lon="Longitude",
                     color="Confirmed", size='size',hover_data=['State/UnionTerritory'],
                     color_continuous_scale='burgyl', animation_frame="Date", 
                     title='Spread total cases over time in India')
fig.update(layout_coloraxis_showscale=True)
fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=3, mapbox_center = {"lat":20.5937,"lon":78.9629})
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import squarify
import plotly_express as px


from IPython.display import Image
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
age_details = pd.read_csv('../input/covid19-in-india/AgeGroupDetails.csv')
india_covid_19 = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
hospital_beds = pd.read_csv('../input/covid19-in-india/HospitalBedsIndia.csv')
individual_details = pd.read_csv('../input/covid19-in-india/IndividualDetails.csv')
#ICMR_details = pd.read_csv('../input/covid19-in-india/ICMRTestingDetails.csv')
ICMR_labs = pd.read_csv('../input/covid19-in-india/ICMRTestingLabs.csv')
state_testing = pd.read_csv('../input/covid19-in-india/StatewiseTestingDetails.csv')


In [ ]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-04-2020.csv')

In [ ]:
india_covid_19['Date'] = pd.to_datetime(india_covid_19['Date'])
state_testing['Date'] = pd.to_datetime(state_testing['Date'])

In [ ]:
cumulative_df = india_covid_19.groupby(["State/UnionTerritory", "Date"])["Confirmed", "Deaths", "Cured"].sum().reset_index()
cumulative_df["Date"] = pd.to_datetime(cumulative_df["Date"] , format="%m/%d/%Y").dt.date
cumulative_df = cumulative_df.sort_values(by="Date").reset_index(drop=True)
start_date = datetime.date(2020, 3, 10)
cumulative_df = cumulative_df[cumulative_df["Date"]>=start_date]
cumulative_df["Date"] = cumulative_df["Date"].astype(str)

fig = px.scatter(cumulative_df, x="Confirmed", y="Deaths", animation_frame="Date", animation_group="State/UnionTerritory",
           size="Confirmed", color="State/UnionTerritory", hover_name="State/UnionTerritory",
           log_x=False, size_max=55, range_x=[0,15000], range_y=[-20,800])

layout = go.Layout(
    title=go.layout.Title(
        text="Changes in number of confirmed & death cases over time in India states",
        x=0.5
    ),
    font=dict(size=14),
    xaxis_title = "Total number of confirmed cases",
    yaxis_title = "Total number of death cases"
)

fig.update_layout(layout)

fig.show()

In [ ]:
import plotly.express as px

fig = px.pie(age_details, values='TotalCases', names='AgeGroup',title='Confirmed cases of India')
fig.show()

In [ ]:
labels = ['Missing', 'Male', 'Female']
sizes = []
sizes.append(individual_details['gender'].isnull().sum())
sizes.append(list(individual_details['gender'].value_counts())[0])
sizes.append(list(individual_details['gender'].value_counts())[1])

explode = (0, 0.1, 0)
colors = ['#ffcc99','#66b3ff','#ff9999']

plt.figure(figsize= (15,10))
plt.title('Percentage of Gender',fontsize = 20)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',shadow=True, startangle=90)
plt.axis('equal')
plt.tight_layout()



In [ ]:
labels = ['Male', 'Female']
sizes = []
sizes.append(list(individual_details['gender'].value_counts())[0])
sizes.append(list(individual_details['gender'].value_counts())[1])

explode = (0.1, 0)
colors = ['#66b3ff','#ff9999']

plt.figure(figsize= (10,8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90)

plt.title('Percentage of Gender (Ignoring the Missing Values)',fontsize = 20)
plt.axis('equal')
plt.tight_layout()



In [ ]:
dates = list(confirmed_df.columns[4:])
dates = list(pd.to_datetime(dates))
dates_india = dates[8:]

In [ ]:
df1 = confirmed_df.groupby('Country/Region').sum().reset_index()
df2 = deaths_df.groupby('Country/Region').sum().reset_index()
df3 = recovered_df.groupby('Country/Region').sum().reset_index()

k = df1[df1['Country/Region']=='India'].loc[:,'1/30/20':]
india_confirmed = k.values.tolist()[0] 

k = df2[df2['Country/Region']=='India'].loc[:,'1/30/20':]
india_deaths = k.values.tolist()[0] 

k = df3[df3['Country/Region']=='India'].loc[:,'1/30/20':]
india_recovered = k.values.tolist()[0] 

plt.figure(figsize= (15,10))
plt.xticks(rotation = 90 ,fontsize = 11)
plt.yticks(fontsize = 10)
plt.xlabel("Dates",fontsize = 20)
plt.ylabel('Total cases',fontsize = 20)
plt.title("Total Confirmed, Active, Death in India" , fontsize = 20)

ax1 = plt.plot_date(y= india_confirmed,x= dates_india,label = 'Confirmed',linestyle ='-',color = 'b')
ax2 = plt.plot_date(y= india_recovered,x= dates_india,label = 'Recovered',linestyle ='-',color = 'g')
ax3 = plt.plot_date(y= india_deaths,x= dates_india,label = 'Death',linestyle ='-',color = 'r')
plt.legend();



In [ ]:
df = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
data = df.copy()
data['Date'] = data['Date'].apply(pd.to_datetime)
data.drop(['Sno', 'Time'],axis=1,inplace=True)

# collect present data
from datetime import date
data_apr = data[data['Date'] > pd.Timestamp(date(2020,4,12))]

# prepaing data state wise
state_cases = data_apr.groupby('State/UnionTerritory')['Confirmed','Deaths','Cured'].max().reset_index()
state_cases['Active'] = state_cases['Confirmed'] - (state_cases['Deaths']- state_cases['Cured'])
state_cases["Death Rate (per 100)"] = np.round(100*state_cases["Deaths"]/state_cases["Confirmed"],2)
state_cases["Cure Rate (per 100)"] = np.round(100*state_cases["Cured"]/state_cases["Confirmed"],2)

In [ ]:
state_details = pd.pivot_table(df, values=['Confirmed','Deaths','Cured'], index='State/UnionTerritory', aggfunc='max')
state_details['Recovery Rate'] = round(state_details['Cured'] / state_details['Confirmed'],2)
state_details['Deaths']['Madhya Pradesh#']=119

state_details['Deaths']=state_details['Deaths'].astype(np.float32)
#state_details=state_details.reset_index()
state_details['Active']=state_details['Confirmed']-state_details['Cured']-state_details['Deaths']

state_details['Death Rate'] = round(state_details['Deaths'] /state_details['Confirmed'], 2)
state_details = state_details.sort_values(by='Confirmed', ascending= False)
#state_details.style.background_gradient(cmap='PuBuGn')
state_details.style.bar(subset=['Confirmed'], color='#FDD017')\
                    .bar(subset=['Cured'], color='lime')\
                    .bar(subset=['Deaths'], color='red')\
                    .bar(subset=['Active'], color='#0000FF')\
                    .bar(subset=['Recovery Rate'], color='#B1FB17')\
                    .bar(subset=['Death Rate'], color='#C0C0C0')

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/COVID-19_India_Total_Cases_Animated_Map.gif/527px-COVID-19_India_Total_Cases_Animated_Map.gif)

In [ ]:
state_testing = pd.read_csv('../input/covid19-in-india/StatewiseTestingDetails.csv')
state_testing

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/CSIR_Corona_Combat_Mission.png/463px-CSIR_Corona_Combat_Mission.png)

# Testings

In [ ]:
labs = pd.read_csv("../input/covid19-in-india/ICMRTestingLabs.csv")
fig = px.treemap(labs, path=['state','city'],
                  color='city', hover_data=['lab','address'],
                  color_continuous_scale='reds')
fig.show()

In [ ]:
testing=state_testing.groupby('State').sum().reset_index()
testing=testing.sort_values(['TotalSamples'], ascending=True)
fig = px.bar(testing, 
             x="TotalSamples",
             y="State", 
             orientation='h',
             height=800,
             title='Testing statewise insight')
fig.show()

# Laboratory

In [ ]:
import plotly.express as px
values = list(ICMR_labs['state'].value_counts())
names = list(ICMR_labs['state'].value_counts().index)
df = pd.DataFrame(list(zip(values, names)), 
               columns =['values', 'names'])

fig = px.bar(df, 
             x="values", 
             y="names", 
             orientation='h',
             height=1000,
             title='ICMR Testing Centers in each State')
fig.show()


# Hospital Bed

In [ ]:
plt.figure(figsize=(20,60))
plt.subplot(4,1,1)
hospital_beds=hospital_beds.sort_values('NumUrbanHospitals_NHP18', ascending= False)
sns.barplot(data=hospital_beds,y='State/UT',x='NumUrbanHospitals_NHP18',color=sns.color_palette('RdBu')[0])
plt.title('Urban Hospitals per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospital_beds.shape[0]):
    count = hospital_beds.iloc[i]['NumUrbanHospitals_NHP18']
    plt.text(count+10,i,count,ha='center',va='center')

plt.subplot(4,1,2)
hospital_beds=hospital_beds.sort_values('NumRuralHospitals_NHP18', ascending= False)
sns.barplot(data=hospital_beds,y='State/UT',x='NumRuralHospitals_NHP18',color=sns.color_palette('RdBu')[1])
plt.title('Rural Hospitals per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospital_beds.shape[0]):
    count = hospital_beds.iloc[i]['NumRuralHospitals_NHP18']
    plt.text(count+100,i,count,ha='center',va='center')

plt.subplot(4,1,3)
hospitalBeds=hospital_beds.sort_values('NumUrbanBeds_NHP18', ascending= False)
sns.barplot(data=hospitalBeds,y='State/UT',x='NumUrbanBeds_NHP18',color=sns.color_palette('RdBu')[5])
plt.title('Rural Beds per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospitalBeds.shape[0]):
    count = hospitalBeds.iloc[i]['NumUrbanBeds_NHP18']
    plt.text(count+1500,i,count,ha='center',va='center')

plt.subplot(4,1,4)
hospitalBeds=hospitalBeds.sort_values('NumRuralBeds_NHP18', ascending= False)
sns.barplot(data=hospitalBeds,y='State/UT',x='NumRuralBeds_NHP18',color=sns.color_palette('RdBu')[3])
plt.title('Rural Beds per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospitalBeds.shape[0]):
    count = hospitalBeds.iloc[i]['NumRuralBeds_NHP18']
    plt.text(count+1500,i,count,ha='center',va='center')

plt.show()
plt.tight_layout()


# Containment Zones

In [ ]:
zone=pd.read_csv('/kaggle/input/covid-19-india-zone-classification/lockdownindiawarningzones.csv')
zone.style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

In [ ]:
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]}')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
    
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
plotPerColumnDistribution(zone, 10, 5)

In [ ]:
fig = px.treemap(zone, path=["State",'Zone','District'],
                  color='Zone',
                 color_discrete_map={'(?)':'black','Green Zone':'#00FF00', 'Red Zone':'#ff0000', 'Orange Zone':'#FFA500'},
                 hover_data=['Zone'])
fig.show()

# State view: 

In [ ]:
country_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
country_codes = country_codes.drop('GDP (BILLIONS)', 1)
country_codes.rename(columns={'COUNTRY': 'Country', 'CODE': 'Code'}, inplace=True)

In [ ]:
virus_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

prev_index = 0
first_time = False
tmp = 0


for i, row in virus_data.iterrows():

    if(virus_data.loc[i,'SNo'] < 1342 and virus_data.loc[i,'Province/State']=='Hubei'):
        if(first_time):
            tmp = virus_data.loc[i,'Confirmed']
            prev_index = i
            virus_data.loc[i,'Confirmed'] = virus_data.loc[i,'Confirmed'] + 593
            first_time = False
        else:
            increment = virus_data.loc[i,'Confirmed'] - tmp
            tmp = virus_data.loc[i,'Confirmed']
            virus_data.loc[i,'Confirmed'] = virus_data.loc[prev_index,'Confirmed'] + increment + 593
            prev_index = i
    

virus_data.rename(columns={'Country/Region': 'Country', 'ObservationDate': 'Date'}, inplace=True)
virus_data = virus_data.fillna('unknow')
virus_data['Country'] = virus_data['Country'].str.replace('US','United States')
virus_data['Country'] = virus_data['Country'].str.replace('UK','United Kingdom') 
virus_data['Country'] = virus_data['Country'].str.replace('Mainland China','China')
virus_data['Country'] = virus_data['Country'].str.replace('South Korea','Korea, South')
virus_data['Country'] = virus_data['Country'].str.replace('North Korea','Korea, North')
virus_data['Country'] = virus_data['Country'].str.replace('Macau','China')
virus_data['Country'] = virus_data['Country'].str.replace('Ivory Coast','Cote d\'Ivoire')
virus_data = pd.merge(virus_data,country_codes,on=['Country'])
#virus_data.head()
#print(len(virus_data))

In [ ]:
covid_India_cases = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
covid_India_cases=covid_India_cases.dropna()
covid_India_cases.rename(columns={'State/UnionTerritory': 'State', 'Cured': 'Recovered', 'Confirmed': 'Confirmed'}, inplace=True)

covid_India_cases = covid_India_cases.fillna('unknow')
top_country = covid_India_cases.loc[covid_India_cases['Date'] == covid_India_cases['Date'].iloc[-1]]
top_country = top_country.groupby(['State'])['Confirmed'].sum().reset_index()
top_country = top_country.sort_values('Confirmed', ascending=False)
top_country = top_country[:30]
top_country_codes = top_country['State']
top_country_codes = list(top_country_codes)

#countries = virus_data.loc[virus_data['Country'] in top_country_codes]
countries = covid_India_cases[covid_India_cases['State'].isin(top_country_codes)]
countries_day = countries.groupby(['Date','State'])['Confirmed','Deaths','Recovered'].sum().reset_index()


exponential_line_x = []
exponential_line_y = []
for i in range(16):
    exponential_line_x.append(i)
    exponential_line_y.append(i)
    
################################    Maharashtra    #################
Maharashtra = countries_day.loc[countries_day['State']=='Maharashtra']
Maharashtra=Maharashtra.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Maharashtra = []
new_confirmed_cases_Maharashtra.append( list(Maharashtra['Confirmed'])[0] - list(Maharashtra['Deaths'])[0] 
                           - list(Maharashtra['Recovered'])[0] )

for i in range(1,len(Maharashtra)):

    new_confirmed_cases_Maharashtra.append( list(Maharashtra['Confirmed'])[i] - 
                                     list(Maharashtra['Deaths'])[i] - 
                                     list(Maharashtra['Recovered'])[i])
    
#######################   Gujarat   ############
Gujarat = countries_day.loc[countries_day['State']=='Gujarat']
Gujarat=Gujarat.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Gujarat = []
new_confirmed_cases_Gujarat.append( list(Gujarat['Confirmed'])[0] - list(Gujarat['Deaths'])[0] 
                           - list(Gujarat['Recovered'])[0] )

for i in range(1,len(Gujarat)):
    
    new_confirmed_cases_Gujarat.append( list(Gujarat['Confirmed'])[i] - 
                                  list(Gujarat['Deaths'])[i] - 
                                  list(Gujarat['Recovered'])[i])
    
###########################    Delhi   ##################3
Delhi = countries_day.loc[countries_day['State']=='Delhi']
Delhi=Delhi.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Delhi = []
new_confirmed_cases_Delhi.append( list(Delhi['Confirmed'])[0] - list(Delhi['Deaths'])[0] 
                           - list(Delhi['Recovered'])[0] )

for i in range(1,len(Delhi)):
    
    new_confirmed_cases_Delhi.append( list(Delhi['Confirmed'])[i] - 
                                     list(Delhi['Deaths'])[i] - 
                                    list(Delhi['Recovered'])[i])
    
#############################    Madhya Pradesh    ################3
Madhya_Pradesh = countries_day.loc[countries_day['State']=='Madhya Pradesh']
Madhya_Pradesh=Madhya_Pradesh.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Madhya_Pradesh = []
new_confirmed_cases_Madhya_Pradesh.append( list(Madhya_Pradesh['Confirmed'])[0] - list(Madhya_Pradesh['Deaths'])[0] 
                           - list(Madhya_Pradesh['Recovered'])[0] )

for i in range(1,len(Madhya_Pradesh)):
    
    new_confirmed_cases_Madhya_Pradesh.append( list(Madhya_Pradesh['Confirmed'])[i] - 
                                     list(Madhya_Pradesh['Deaths'])[i] - 
                                    list(Madhya_Pradesh['Recovered'])[i])
    
################################   Rajasthan  ##########
Rajasthan = countries_day.loc[countries_day['State']=='Rajasthan']
Rajasthan=Rajasthan.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Rajasthan = []
new_confirmed_cases_Rajasthan.append( list(Rajasthan['Confirmed'])[0] - list(Rajasthan['Deaths'])[0] 
                           - list(Rajasthan['Recovered'])[0] )

for i in range(1,len(Rajasthan)):
    
    new_confirmed_cases_Rajasthan.append( list(Rajasthan['Confirmed'])[i] - 
                                     list(Rajasthan['Deaths'])[i] - 
                                    list(Rajasthan['Recovered'])[i])
    
#################################    Uttar Pradesh   ##############
Uttar_Pradesh = countries_day.loc[countries_day['State']=='Uttar Pradesh']
Uttar_Pradesh=Uttar_Pradesh.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Uttar_Pradesh = []
new_confirmed_cases_Uttar_Pradesh.append( list(Uttar_Pradesh['Confirmed'])[0] - list(Uttar_Pradesh['Deaths'])[0] 
                           - list(Uttar_Pradesh['Recovered'])[0] )

for i in range(1,len(Uttar_Pradesh)):
    
    new_confirmed_cases_Uttar_Pradesh.append( list(Uttar_Pradesh['Confirmed'])[i] - 
                                     list(Uttar_Pradesh['Deaths'])[i] - 
                                    list(Uttar_Pradesh['Recovered'])[i])
    
#####################################  Tamil Nadu  ############
Tamil_Nadu = countries_day.loc[countries_day['State']=='Tamil Nadu']
Tamil_Nadu=Tamil_Nadu.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Tamil_Nadu = []
new_confirmed_cases_Tamil_Nadu.append( list(Tamil_Nadu['Confirmed'])[0] - list(Tamil_Nadu['Deaths'])[0] 
                           - list(Tamil_Nadu['Recovered'])[0] )

for i in range(1,len(Tamil_Nadu)):
    
    new_confirmed_cases_Tamil_Nadu.append( list(Tamil_Nadu['Confirmed'])[i] - 
                                     list(Tamil_Nadu['Deaths'])[i] - 
                                    list(Tamil_Nadu['Recovered'])[i])
######################################## Andhra Pradesh ##############
Andhra_Pradesh = countries_day.loc[countries_day['State']=='Andhra Pradesh']
Andhra_Pradesh=Andhra_Pradesh.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Andhra_Pradesh = []
new_confirmed_cases_Andhra_Pradesh.append( list(Andhra_Pradesh['Confirmed'])[0] - list(Andhra_Pradesh['Deaths'])[0] 
                           - list(Andhra_Pradesh['Recovered'])[0] )

for i in range(1,len(Andhra_Pradesh)):
    
    new_confirmed_cases_Andhra_Pradesh.append( list(Andhra_Pradesh['Confirmed'])[i] - 
                                     list(Andhra_Pradesh['Deaths'])[i] - 
                                    list(Andhra_Pradesh['Recovered'])[i])

########################################Telengana#####################

#Telengana = countries_day.loc[countries_day['State']=='Telengana']
#Telengana=Telengana.sort_values('Confirmed',ascending=True)

#new_confirmed_cases_Telengana = []
#new_confirmed_cases_Telengana.append( list(Telengana['Confirmed'])[0] - list(Telengana['Deaths'])[0] 
#                           - list(Telengana['Recovered'])[0] )

#for i in range(1,len(Telengana)):
    
#    new_confirmed_cases_Telengana.append( list(Telengana['Confirmed'])[i] - 
#                                     list(Telengana['Deaths'])[i] - 
#                                    list(Telengana['Recovered'])[i])


##########################################  West Bengal #####################33
West_Bengal = countries_day.loc[countries_day['State']=='West Bengal']
West_Bengal=West_Bengal.sort_values('Confirmed',ascending=True)

new_confirmed_cases_West_Bengal = []
new_confirmed_cases_West_Bengal.append( list(West_Bengal['Confirmed'])[0] - list(West_Bengal['Deaths'])[0] 
                           - list(West_Bengal['Recovered'])[0] )

for i in range(1,len(West_Bengal)):
    
    new_confirmed_cases_West_Bengal.append( list(West_Bengal['Confirmed'])[i] - 
                                     list(West_Bengal['Deaths'])[i] - 
                                    list(West_Bengal['Recovered'])[i])
############################################ 
p1 = figure(plot_width=800, plot_height=550, title="Trajectory of Covid-19")
p1.grid.grid_line_alpha=0.3
p1.ygrid.band_fill_color = "olive"
p1.ygrid.band_fill_alpha = 0.1
p1.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p1.yaxis.axis_label = 'New confirmed cases (Log scale)'

p1.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=0.5)

p1.line(np.log(list(Maharashtra['Confirmed'])), np.log(new_confirmed_cases_Maharashtra), color='#DBAE23', 
        legend_label='Maharashtra', line_width=1)
p1.circle(np.log(list(Maharashtra['Confirmed'])[-1]), np.log(new_confirmed_cases_Maharashtra[-1]), fill_color="white", size=5)

p1.line(np.log(list(Gujarat['Confirmed'])), np.log(new_confirmed_cases_Gujarat), color='#3EC358', 
        legend_label='Gujarat', line_width=1)
p1.circle(np.log(list(Gujarat['Confirmed'])[-1]), np.log(new_confirmed_cases_Gujarat[-1]), fill_color="white", size=5)

p1.line(np.log(list(Delhi['Confirmed'])), np.log(new_confirmed_cases_Delhi), color='#C3893E', 
       legend_label='Delhi', line_width=1)
p1.circle(np.log(list(Delhi['Confirmed'])[-1]), np.log(new_confirmed_cases_Delhi[-1]), fill_color="white", size=5)


p1.line(np.log(list(Madhya_Pradesh['Confirmed'])), np.log(new_confirmed_cases_Madhya_Pradesh), color='#3E4CC3', 
        legend_label='Madhya Pradesh', line_width=1)
p1.circle(np.log(list(Madhya_Pradesh['Confirmed'])[-1]), np.log(new_confirmed_cases_Madhya_Pradesh[-1]), fill_color="white", size=5)

p1.line(np.log(list(Rajasthan['Confirmed'])), np.log(new_confirmed_cases_Rajasthan), color='#F54138', 
        legend_label='Rajasthan', line_width=1)
p1.circle(np.log(list(Rajasthan['Confirmed'])[-1]), np.log(new_confirmed_cases_Rajasthan[-1]), fill_color="white", size=5)

p1.line(np.log(list(Uttar_Pradesh['Confirmed'])), np.log(new_confirmed_cases_Uttar_Pradesh), color='#23BCDB', 
        legend_label='Uttar Pradesh', line_width=1)
p1.circle(np.log(list(Uttar_Pradesh['Confirmed'])[-1]), np.log(new_confirmed_cases_Uttar_Pradesh[-1]), fill_color="white", size=5)

p1.line(np.log(list(Tamil_Nadu['Confirmed'])), np.log(new_confirmed_cases_Tamil_Nadu), color='#010A0C', 
        legend_label='Tamil Nadu', line_width=1)
p1.circle(np.log(list(Tamil_Nadu['Confirmed'])[-1]), np.log(new_confirmed_cases_Tamil_Nadu[-1]), fill_color="white", size=5)

p1.line(np.log(list(Andhra_Pradesh['Confirmed'])), np.log(new_confirmed_cases_Andhra_Pradesh), color='#bf40bf', 
        legend_label='Andhra Pradesh', line_width=1)
p1.circle(np.log(list(Andhra_Pradesh['Confirmed'])[-1]), np.log(new_confirmed_cases_Andhra_Pradesh[-1]), fill_color="white", size=5)

#p1.line(np.log(list(Telengana['Confirmed'])), np.log(new_confirmed_cases_Telengana), color='lime', 
#        legend_label='Telengana', line_width=1)
#p1.circle(np.log(list(Telengana['Confirmed'])[-1]), np.log(new_confirmed_cases_Telengana[-1]), fill_color="white", size=5)


p1.line(np.log(list(West_Bengal['Confirmed'])), np.log(new_confirmed_cases_West_Bengal), color='#0000ff', 
        legend_label='West Bengal', line_width=1)
p1.circle(np.log(list(West_Bengal['Confirmed'])[-1]), np.log(new_confirmed_cases_West_Bengal[-1]), fill_color="white", size=5)



p1.legend.location = "bottom_right"

output_file("coronavirus.html", title="coronavirus.py")

show(p1)

* **Looks like Telengana is goint into a safe zone**
* **Here comes the most controversial state - "West Bengal" along with the most infected state - "Maharashtra":**

In [ ]:
p1 = figure(plot_width=800, plot_height=550, title="Trajectory of Covid-19")
p1.grid.grid_line_alpha=0.3
p1.ygrid.band_fill_color = "olive"
p1.ygrid.band_fill_alpha = 0.1
p1.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p1.yaxis.axis_label = 'New confirmed cases (Log scale)'

p1.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=0.5)

p1.line(np.log(list(Maharashtra['Confirmed'])), np.log(new_confirmed_cases_Maharashtra), color='#DBAE23', 
        legend_label='Maharashtra', line_width=1)
p1.circle(np.log(list(Maharashtra['Confirmed'])[-1]), np.log(new_confirmed_cases_Maharashtra[-1]), fill_color="white", size=5)

p1.line(np.log(list(West_Bengal['Confirmed'])), np.log(new_confirmed_cases_West_Bengal), color='Blue', 
        legend_label='West_Bengal', line_width=1)
p1.circle(np.log(list(West_Bengal['Confirmed'])[-1]), np.log(new_confirmed_cases_West_Bengal[-1]), fill_color="white", size=5)



p1.legend.location = "bottom_right"

output_file("coronavirus.html", title="coronavirus.py")

show(p1)

# India COVID19 Forecasting

# Sigmoid model

I thought of a sigmoidal function because China's data resembled a sigmoidal shape. Therefore, I try to fit sigmoid functions onto India's. Its just a guess as per the graph about the cases in India.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Any results you write to the current directory are saved as output.
train=pd.read_csv('/kaggle/input/coronavirus-2019ncov/covid-19-all.csv')

In [ ]:
train.head(5)

In [ ]:
country_df = train[train['Country/Region']=='India'].groupby('Date')['Confirmed','Deaths'].sum()
country_df['day_count'] = list(range(1,len(country_df)+1))
ydata = country_df.Confirmed
xdata = country_df.day_count
country_df['rate'] = (country_df.Confirmed-country_df.Confirmed.shift(1))/country_df.Confirmed
country_df['increase'] = (country_df.Confirmed-country_df.Confirmed.shift(1))

plt.plot(xdata, ydata, 'o')
plt.title("India")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()

In [ ]:
from scipy.optimize import curve_fit
import pylab


def sigmoid(x,c,a,b):
     y = c*1 / (1 + np.exp(-a*(x-b)))
     return y
#country_df.ConfirmedCases
#country_df.day_count
xdata = np.array([1, 2, 3,4, 5, 6, 7])
ydata = np.array([0, 0, 13, 35, 75, 89, 91])

#([low_a,low_b],[high_a,high_b])
#low x --> low b
#high y --> high c
#a is the sigmoidal shape.
popt, pcov = curve_fit(sigmoid, xdata, ydata, method='dogbox',bounds=([0.,0., 0.],[100,2, 10.]))
print(popt)

x = np.linspace(-1, 10, 50)
y = sigmoid(x, *popt)

pylab.plot(xdata, ydata, 'o', label='data')
pylab.plot(x,y, label='fit')
pylab.ylim(-0.05, 105)
pylab.legend(loc='best')
pylab.show()


****Sigmoid function,

Here is a snap of how I learnt to fit Sigmoid Function - y = c/(1+np.exp(-a*(x-b))) and 3 coefficients [c, a, b]:

* c - the maximum value (eventual maximum infected people, the sigmoid scales to this value eventually)
* a - the sigmoidal shape (how the infection progress. The smaller, the softer the sigmoidal shape is)
* b - the point where sigmoid start to flatten from steepening (the midpoint of sigmoid, when the rate of increase start to slow down)



In [ ]:
in_df = train[train['Country/Region']=='India'].groupby('Date')['Confirmed','Deaths','Recovered'].sum().reset_index(False)
in_df['Active']=in_df['Confirmed']-in_df['Deaths']-in_df['Recovered']
in_df = in_df[in_df.Active>=100]

In [ ]:
in_df.tail()

In [ ]:
from scipy.optimize import curve_fit
import pylab
from datetime import timedelta

in_df['day_count'] = list(range(1,len(in_df)+1))
in_df['increase'] = (in_df.Active-in_df.Active.shift(1))
in_df['rate'] = (in_df.Active-in_df.Active.shift(1))/in_df.Active


def sigmoid(x,c,a,b):
     y = c*1 / (1 + np.exp(-a*(x-b)))
     return y

xdata = np.array(list(in_df.day_count)[::2])
ydata = np.array(list(in_df.Active)[::2])

population=1.332*10**9
#popt, pcov = curve_fit(sigmoid, xdata, ydata, method='dogbox',bounds=([0.,0., 0.],[population,6, 100.]))
#print(popt)

In [ ]:
est_a = 197000 #popt[0]
est_b = 0.07 #popt[1]
est_c = 76 #popt[2]
x = np.linspace(-1, in_df.day_count.max()+50, 50)
y = sigmoid(x,est_a,est_b,est_c)
pylab.plot(xdata, ydata, 'o', label='data')
pylab.plot(x,y, label='fit',alpha = 0.6)
pylab.ylim(-0.05, est_a*1.05)
pylab.xlim(-0.05, est_c*2.05)
pylab.legend(loc='best')
plt.xlabel('days from day 1')
plt.ylabel('confirmed cases')
plt.title('India')
pylab.show()


#print('model start date:',in_df[in_df.day_count==1].index[0])
#print('model start infection:',int(in_df[in_df.day_count==1].Active[0]))
print('model fitted max Active at:',int(est_a))
print('model sigmoidal coefficient is:',round(est_b,3))
print('model curve stop steepening, start flattening by day:',int(est_c))
print('model curve flattens by day:',int(est_c)*2)
display(in_df.head(3))
display(in_df.tail(3))

## From this, its seen that in case of India if the graph goes like that:
*  **max Active case: 197000**  ☠️
*  **curve stop steepening, start flattening by day: 76 ,which is: 29/05/2020**
*  **curve flattens by day: 152  which is: 12/08/2020**

![](https://media0.giphy.com/media/WOkuHrhNv4LQVeSoik/giphy.gif?cid=ecf05e4748c6ef5d1983f3a0b50e6195a51aa49635340b4c&rid=giphy.gif)

# Hope you guys find this notebook useful. 
# If you like this notebook, please upvote.***👍 
# Thanks in advance**